## algorithm design and anlysis-2025 spring  homework 3
**Deadline**：2025.5.14

**name**:罗雨欣


note：
---
1. 带有\*的题目，申请免上课的同学，必须完成，其他同学选作；
2. 请独立完成，如求助了他人或者大模型，请著明，并且不可省略算法分析部分；
4. 如若作答有雷同，全部取消成绩；
3. 需要书面作答的题目，可以通过引用图片的形式添加，但是注意上传项目时包含所引用的图片的源文件；
4. $log_n$ 默认表示$log_2{n}$;

## 问题 1  

请证明基于比较的排序算法计算复杂度下界为$\Omega(nlogn)$

you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：   
证明如下：   
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3-1.jpg"> </div>

## 问题 2 

构建二叉搜索树

> 1. 给定一个数组 [48, 33, 49, 47, 42, 46, 32], 构建一个二叉搜索树存储这些数据，请**绘制**所构建的**二叉搜索树**（标明结点的值）。
> 2. 从第1问的二叉搜索树中删除33，请**绘制**删除33后的**二叉搜索树**（标明结点的值）

- 推荐使用graphviz包绘制树的结构

In [9]:
import graphviz
import os
dot_path = r"D:\\Environment\\Graphviz\\bin"
#添加到系统 PAT
os.environ["PATH"] += os.pathsep + dot_path  
class TreeNode:
    def __init__(self,val=0,left=None,right=None):
        self.val=val
        self.left=left
        self.right=right
def InsertNode(root,val):
    #若不存在根节点 插入根节点 
    if not root:
        root=TreeNode(val)
    #若存在则递归判断 大于当前所在位置则往左插判断
    else:
        if val<root.val:
            root.left=InsertNode(root.left,val)
        else:
            root.right=InsertNode(root.right,val)
    return root
def FindMin(node):
    while (node.left):
        node=node.left
    return node
def DeleteNode(root,key):
    #如果寻找的value<根节点值
    if key<root.val:
        root.left=DeleteNode(root.left,key)
     #如果寻找的value>根节点值
    elif key>root.val:
        root.right=DeleteNode(root.right,key)
    else:
        if not root.left:
            return root.right
        elif not root.right:
            return root.left
        #都存在 递归找根右子节点最小节点 找后继
        else:
            #swapnode为后继
            swapnode=FindMin(root.right)
            #删除它第一步是需要把值给交换了 左右子节点无变化
            root.val=swapnode.val
            #第二步 把原本swapnode那个位置的节点删了 从root.right开始重构
            root.right=DeleteNode(root.right,swapnode.val)
    return root
def GenerateTree(TreeValues):
    root=None
    for TreeValue in TreeValues:
        root=InsertNode(root,TreeValue)
    return root
def DrawTree(root, filename):
    dot = graphviz.Digraph(comment="Binary Search Tree")
    def AddNodesEdges(node):
        if not node:
            return
        dot.node(str(id(node)),str(node.val))
        if node.left:
            #创造左子节点
            dot.node(str(id(node.left)),str(node.left.val))
            #添加一条从当前节点到左子节点的边 使用标签指明是左子节点
            dot.edge(str(id(node)),str(id(node.left)),label="L")
            #递归添加左子树
            AddNodesEdges(node.left)
        if node.right:
            dot.node(str(id(node.right)),str(node.right.val))
            dot.edge(str(id(node)),str(id(node.right)),label="R")
            AddNodesEdges(node.right)
    AddNodesEdges(root)
    dot.render(filename, format="png", cleanup=True)
    print(f"树图已保存为：{filename}.png")
TreeValues=[48, 33, 49, 47, 42, 46, 32]
root=GenerateTree(TreeValues)
DrawTree(root, "/fig/bst_before_delete")
root=DeleteNode(root,32)
DrawTree(root, "/fig/bst_after_delete")

树图已保存为：/fig/bst_before_delete.png
树图已保存为：/fig/bst_after_delete.png


you can refer a handwritten picture, pleas upload the picture in /fig/xxx.png
answer：   
运行结果如下：   
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3-2.jpg"> </div>
删除前的二叉树为：   
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/bst_before_delete.png"> </div>
删除后的二叉树为：   
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/bst_after_delete.png"> </div>

## 问题 3 

> 下面是九门课程的开始和结束时间:
>
> [(9:00,12:30), (11:00,14:00), (13:00, 14:30), (9:00,10:30),(13:00, 14:30),(14:00,16:30), (15:00,16:30), (15:00,16:30), (9:00,10:30)]
>
> 请使用贪婪算法为这九门课分配教室，要求在同一天内安排这些课，并且每个教室同一时间只能安排一门课。

**请问最少需要几间教室，罗列出每个教室安排的课程**

idea：
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3-3.png"> </div>
后面实验发现也可以不用堆 可以用数组来存储每个教室最后的结束时间也节省了空间  但是要注意一下关于时间对比的时候要转换类型为datetime方便比较  

In [25]:
from datetime import datetime

def AllocateRoom(schedule):
    #先要对时间表排序 按开始时间排序 开始时间一样按结束时间排序
    sorted_schedule=sorted(schedule,key=lambda x: (datetime.strptime(x[0],"%H:%M"), datetime.strptime(x[1],"%H:%M")))
    #print(sorted_schedule)
    #rooms存储每个教室的最后结束时间
    rooms=[]
    assignments=[]
    for start_time,end_time in sorted_schedule:
        assign_tag=False
        #尝试分配到现有教室 不够就新安排教室
        for i,last_end_time in enumerate(rooms):
            #另外的开始的时间慢于现存的教室的结束时间可以分配
            if datetime.strptime(start_time,"%H:%M") >= datetime.strptime(last_end_time,"%H:%M"):
                rooms[i]=end_time
                #把分配好的教室给加入对应教室分配时间表
                assignments[i].append((start_time, end_time))
                assign_tag=True
                break
        #无可用教室 新增
        if not assign_tag:
            rooms.append(end_time)
            assignments.append([start_time, end_time])
    return len(rooms), assignments
schedule=[
    ("9:00","12:30"),("11:00","14:00"),("13:00","14:30"),
    ("9:00","10:30"),("13:00","14:30"),("14:00","16:30"),
    ("15:00","16:30"),("15:00","16:30"),("9:00","10:30")
]
num_rooms,room_assignments=AllocateRoom(schedule)
print(f"最少需要 {num_rooms} 间教室")
for i,assignment in enumerate(room_assignments, 1):
     print(f"教室 {i}: {assignment}")


最少需要 3 间教室
教室 1: ['9:00', '10:30', ('11:00', '14:00'), ('14:00', '16:30')]
教室 2: ['9:00', '10:30', ('13:00', '14:30'), ('15:00', '16:30')]
教室 3: ['9:00', '12:30', ('13:00', '14:30'), ('15:00', '16:30')]


## 问题 4 

> 爬楼梯问题：假设爬楼梯时你每次只能爬一阶或者爬两阶，问爬上n阶的楼梯，你一共有多少种方法

请设计算法完成该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：  
动态规划问题 分解问题 爬上第n阶楼梯有俩种方案   
从第n-1阶上爬一阶  
从第n-2阶上爬两阶  
故f(n)=f(n-1)+f(n-2) 等于爬到第n-1阶方案加爬到第n-2方案   
分解...直到f(1)=1 f(0)=1   
用一阶数组就可以存储所有情况
f(0)=1 f(1)=1 f(2)=f(0)+f(1) f(3)=f(2)+f(1)......f(n)=f(n-1)+f(n-2)   
时间复杂度：$O(n)$  

In [26]:
# add your code here
# your algorithm time complexity is: 
def clim_stairs(n):
    f=[0]*(n+1)
    f[0]=f[1]=1
    for i in range (2,n+1):
        f[i]=f[i-1]+f[i-2]
    return f[n]
n=5
print(f"爬上{n}阶的楼梯，一共有{clim_stairs(n)}种方法")

爬上5阶的楼梯，一共有8种方法


## 问题 5 

> 0-1背包问题：现在有4块大理石，每个大理石对应的重量和价值使用一个元组表示, 即（重量，价值），4块大理石的重量和价值为：[(5,10), (4,40), (6,30), (3, 50)], 假设你有一辆最大承重为9的小推车，请问使用这个小推车装大理石的最大价值为多少

请设计一个算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：  
可以抽象为0-1背包问题  
记f(k,w)为推车容量为w时，现在有k块大理石可装，所以能承载的最大的价值
状态方程为：  
f(k,w)=max{f(k-1,w-wk)+vk,f(k-1,w)}  当wk<=w时  (wk为第k块大理石的重量，vk为第k块大理石的价值)   
f(k,w)=f(k-1,w)  当wk>w时     
由图  
<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3-5.png"> </div>
可由上观察到要运用到二维数组  
时间复杂度：$O(k \times W)$，其中 $k$ 是物品数，$W$ 是最大容量

In [19]:
# add your code here
# your algorithm time complexity is:O(k✖W)，其中k是物品数，W是最大容量
def knapsack(items,max_weight):
    #计算物品数量
    n=len(items)
    #初始化二维数组表f[k+1][max_weight+1]
    f=[[0]*(max_weight+1) for i in range (n+1)]
    #开始填表 第一行全为0（f[0][]=0）所以从第二行开始也就是[1:n] [1:n+1)
    for k in range (1,n+1):
        #从第二列开始
        for w in range (1,max_weight+1):
            if(items[k-1][0]>w):
                f[k][w]=f[k-1][w]
            else:
                f[k][w]=max((f[k-1][w-items[k-1][0]]+items[k-1][1]),f[k-1][w])
    return f[n][max_weight]
        
items=[(5,10),(4,40),(6,30),(3,50)]
max_weight=9
print("最大价值为：", knapsack(items,max_weight)) 

最大价值为： 90


## 问题 6 
给定 `n` 个非负整数表示每个宽度为 `1` 的柱子的高度图，计算按此排列的柱子，下雨之后能接多少雨水。



<div style="text-align:center;width:800px;"> <img alt="" src="./fig/hw3q6.png"> </div>

请分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：
关于储水关键有三方面：
1. 找寻最高左边界 o(n)
2. 找寻最高右边界 o(n)
3. 自己高度
储水量=min{最高左边界,最高右边界}-本身高度   
所以我们只需要找到所有柱子的左边界最高右边界最高再计算即可
最终时间复杂度为：$O(n)$ 

In [39]:
# add your code here
# your algorithm time complexity is:
def CollectRain(height):
    n=len(height)
    #初始化左边界和右边界存储数组
    MaxLeft=[0]*n
    MaxRight=[0]*n
    Collect=[0]*n
    #找寻左边界 注意最左边的那个不需要找了 为0 所以从1开始[1,n-1]  [1,n]
    for i in range (1,n):
        MaxLeft[i]=max(height[i-1],MaxLeft[i-1])
    #print(f"每块地方的最高左边界为{MaxLeft}")
    #找寻右边界 注意从右边开始 最右边的那个右边界不需要再找了 [n-2,0] [n-2,-1)
    for i in range (n-2,-1,-1):
        MaxRight[i]=max(height[i+1],MaxRight[i+1])
    #print(f"每块地方的最高右边界为{MaxRight}")
    #储水量 最左边和最右边可以不用计算 因为没它们俩个的边界[0] [n-1] 则从[1,n-2]  [1,n-1)
    for i in range (1,n-1):
        Collect[i]=min(MaxLeft[i],MaxRight[i])-height[i]
    #print(f"每块地方的储水量为为{Collect}")
    return Collect
height1=[0,1,0,2,1,0,1,3,2,1,2,1]
height2=[4,2,0,3,2,5]
Collect1=CollectRain(height1)
Collect2=CollectRain(height2)
All=0
for i in range (1,len(height1)-1):
    if(Collect1[i]>0):
        All=All+Collect1[i]
print(f"height={height1}")
print(f"总储水量为为{All}")
All=0
for i in range (1,len(height2)-1):
    if(Collect2[i]>0):
        All=All+Collect2[i]
print(f"height={height2}")
print(f"总储水量为{All}")

height=[0, 1, 0, 2, 1, 0, 1, 3, 2, 1, 2, 1]
总储水量为为6
height=[4, 2, 0, 3, 2, 5]
总储水量为9


## 问题 7 

**股票投资组合优化**： 假设你是一位投资者，想要在不同的股票中分配你的资金，以最大化你的投资回报。每只股票都有不同的预期收益率和风险。你的目标是选择一些股票，使得总投资金额不超过你的预算，并且预期收益最大化。

在这个情况下，你可以将每只股票视为一个“物品”，其重量为投资金额，价值为预期收益率。然后，你可以使用分级背包问题的方法来选择部分股票，以便在预算内获得最大的预期收益。

以下是一个简化的例子：

假设你有以下三只股票可供选择：

1. 股票 A：投资金额 5000 美元，预期收益率 10%
2. 股票 B：投资金额 3000 美元，预期收益率 8%
3. 股票 C：投资金额 2000 美元，预期收益率 12%

请设计算法找到最优投资方案，分析算法设计思路，计算时间复杂度，并基于python编程实现

idea：   
依照题目所给提示我们可以采用分级背包来选取部分股票    
我们可以先计算它的单位价值 因为可以拆分了 我们选价值高的投资多一点 预算不足以购买整只股票，就只投资部分（即剩余预算）
预算够就买整只，不够就买剩余单位价值高的股票部分    
总时间复杂度：排序时间复杂度为O(nlogn) 选择股票O(n) 所以总的应该是O(nlogn)   

In [67]:
# add your code here
# your algorithm time complexity is:排序时间复杂度为O(nlogn) 选择股票O(n) 所以总的应该是O(nlogn)
class Stock:
    def __init__(self, name, cost, rate):
        self.name=name              
        self.cost=cost 
        self.rate=rate
        self.profit=cost*rate
        #算单位价值
        self.unit_value=self.profit/self.cost 
#股票和预算
def knapsack(stocks,budget):
    #先按照单位价值从高到低排序 
    stocks.sort(key=lambda x:x.unit_value,reverse=True)
    total_profit=0
    Record=[]
    #按单位价值顺序投资 投资价值高的  budget够把该股票能用的cost花完就再换下一个
    for stock in stocks:
        #没钱就不投资了
        if budget<=0:
            break
        #如果预算够 先把单位价值高的allin全麦
        if budget>=stock.cost:
            #收益为投资金额*收益率
            total_profit+=stock.profit
            budget-=stock.cost
            Record.append((stock.name, stock.cost)) 
        #预算不够了就买部分
        else:
            partial_profit=stock.unit_value*budget
            total_profit+=partial_profit
            Record.append((stock.name, budget))
            budget-=budget
    return total_profit,Record
stocks = [
    Stock("A",5000,10*0.01),  
    Stock("B",3000,8*0.01),  
    Stock("C",2000,12*0.01)   
]
#假设9000预算
budget=9000
profit,plan=knapsack(stocks,budget)
print(f"budget为{budget}")
print("投资方案:")
for stock, amount in plan:
    print(f" 投资股票 {stock}: {amount} 美元")
print(f"最大预期收益: {profit}")

budget为9000
投资方案:
 投资股票 C: 2000 美元
 投资股票 A: 5000 美元
 投资股票 B: 2000 美元
最大预期收益: 900.0


## 问题 8

给你二叉搜索树的根节点 root ，该树中的恰好两个节点的值被错误地交换。请在不改变其结构的情况下，恢复这棵树 。设计算法解决该问题，分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

<div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入：root = [1,3,null,null,2]， 输出为：[3,1,null,null,2]， 

解释：3 不能是 1 的左孩子，因为 3 > 1 。交换 1 和 3 使二叉搜索树有效。

2.

 <div style="text-align:center;width:200px;"> <img alt="" src="./fig/hw3q9-1.jpg"> </div>
输入： root=[3,1,4,null,null,2]， 输出为：[2,1,4,null,null,3]

解释：2 不能在 3 的右子树中，因为 2 < 3 。交换 2 和 3 使二叉搜索树有效。

idea：   
二叉搜索树拥有性质在中序遍历之下其为升序 所以可以根据这个来找到存在问题的节点   
中序遍历时前面的应该都小于或等于当前的节点 若不是则逆序了 去找异常的俩个节点交换即可   
时间复杂度主要是在于对于整棵树的中序遍历找异常 所以整体的时间复杂度为:O(n)

In [35]:
# add your code here
# your algorithm time complexity is:O(n)
#先定义好树的结点
class TreeNode:
    def __init__(self,val=0,left=None,right=None):
        self.val=val
        self.left=left
        self.right=right
def InsertNode(root,val):
    #若不存在根节点 插入根节点 
    if not root:
        root=TreeNode(val)
    #若存在则递归判断 大于当前所在位置则往左插判断
    else:
        if val<root.val:
            root.left=InsertNode(root.left,val)
        else:
            root.right=InsertNode(root.right,val)
    return root
def GenerateTree(TreeValues):
    root=None
    for TreeValue in TreeValues:
        root=InsertNode(root,TreeValue)
    return root
def print_inorder(root):
    if not root:
        return
    #左根右
    print_inorder(root.left)
    print(root.val,end=' ')
    print_inorder(root.right)
def recoverTree(root):
    first=second=prev=None
    #中序遍历找错
    def inorder(node):
        nonlocal first,second,prev
        if not node:
            return
        #先遍历左子树
        inorder(node.left)
        #看现在的节点 这里说明逆序了 记录好逆序的地方
        if prev and prev.val>node.val:
            #发现错误记录下来
            if not first:
                first=prev
            second=node
        #没错就更新
        prev=node
        #遍历右子树
        inorder(node.right)
    #调用函数
    inorder(root)
    if first and second:
        first.val, second.val = second.val, first.val
TreeValues=[48, 33, 49, 47, 42, 46, 32]
root=GenerateTree(TreeValues)
print_inorder(root)
#这里手动构建错误 
root.left.val,root.right.val=root.right.val,root.left.val
print('\n'+'制造错误后:')
print_inorder(root)
print('\n'+'恢复:')
recoverTree(root)
print_inorder(root)

32 33 42 46 47 48 49 
制造错误后:
32 49 42 46 47 48 33 
恢复:
32 33 42 46 47 48 49 

## 问题 9  

给你一个二叉树的根节点 root ，树中每个节点都存放有一个 0 到 9 之间的数字。每条从根节点到叶节点的路径都代表一个数字：

> 例如，从根节点到叶节点的路径 1 -> 2 -> 3 表示数字 123 。

设计一个算法计算从根节点到叶节点生成的所有数字之和 ，分析算法设计思路，计算时间复杂度，并基于python编程实现

叶节点:是指没有子节点的节点。

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-1.jpg"> </div>

 输入：root = [1,2,3], 输出：25

解释：从根到叶子节点路径 1->2 代表数字 12，1->3 代表数字 13，因此和为 12 + 13 = 25 。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q10-2.jpg"> </div>

 输入：root = [4,9,0,5,1], 输出：1026

解释：从根到叶子节点路径 4->9->5 代表数字 495，4->9->1 代表数字 491，4->0 代表数字 40。因此和为 495 + 491 + 40 = 1026 。

idea：   
感觉跟hw1最后一题字典树一样也是在深度遍历，就直接深度优先搜索即可采用DFS递归即可  不过字典树不是二叉树 这里指定了二叉树更简单
而且每一步里面 根据深度越深值越来越大 深一级*10 所以每一次都是path_sum*10+node*val 原路径*10+当前节点值 到叶子节点就停止也不用定义结束标识符   
最后递归回溯并加上所有路径数字   
时间复杂度的话每个节点访问一次 n个节点 时间复杂度为：O(n)

In [39]:
# add your code here
# your algorithm time complexity is:O(n)
#先定义好树的结点
class TreeNode:
    def __init__(self,val=0,left=None,right=None):
        self.val=val
        self.left=left
        self.right=right
def SumNumbers(root):
    def dfs(node,path_sum):
        if not node:
            return 0
        path_sum=path_sum*10+node.val
        #到叶子节点就相当于结束了当前路径
        if not node.left and not node.right:
            return path_sum
        #从左右节点深度搜索 直至各自叶子节点
        left_child_sum=dfs(node.left,path_sum)
        right_child_sum=dfs(node.right,path_sum)
        return left_child_sum+right_child_sum
    return dfs(root,0)
#这里没说是完全二叉树 我们就手动构造树 
root=TreeNode(1)
root.left=TreeNode(2)
root.right=TreeNode(3)
print("输入：root = [1,2,3], 输出：",SumNumbers(root))
root2=TreeNode(4)
root2.left=TreeNode(9)
root2.right=TreeNode(0)
root2.left.left=TreeNode(5)
root2.left.right=TreeNode(1)
print("输入：root = [4,9,0,5,1], 输出：",SumNumbers(root2))

输入：root = [1,2,3], 输出： 25
输入：root = [4,9,0,5,1], 输出： 1026


###### 问题 10  

给你一个二叉树的根节点 root ， 检查它是否轴对称。

1. 分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-1.png"> </div>

 输入：root = [1,2,2,3,4,4,3]， 输出：true

解释：二叉树 [1,2,2,3,4,4,3] 对称。

2.

 <div style="text-align:center;width:200px;"> <img alt="figure" src="./fig/hw3q11-2.png"> </div>

 输入：root = [1,2,2,null,3,null,3]， 输出：false

idea   
如果是轴对称的话 我们可以看轴对称树发现：   
对于根节点开始 左右子树值根节点相等 然后左子树的左节点=右子树的右节点 左子树的右节点=右子树的左节点 递归到最后   
每次都要对于根节点的左右子树判断 若空则对称 若只存在一个 不对称 若左右根子树值不痛 不对称 再去判断左右根子树的左节点和右节点 右节点和左节点是否等于   
对于时间复杂度判断：每个节点访问一次 所以为O(n)

In [51]:
# add your code here
# your algorithm time complexity is:O(n)
from collections import deque
#先定义好树的结点
class TreeNode:
    def __init__(self,val=0,left=None,right=None):
        self.val=val
        self.left=left
        self.right=right
def IsSymmetric(root):
    def IsMirror(left,right):
        #如果没有左右子树
        if not left and not right:
            return True
        #左右子树仅存在一个
        if not left or not right:
            return False
        #左右子树都有 开始判断 不等于则返回False
        if left.val!=right.val:
            return False
        return IsMirror(left.left, right.right) and IsMirror(left.right, right.left)
    if not root:
        return True
    return IsMirror(root.left, root.right)
        #if not IsSymmetric(root.left):
            #return False
        #if not IsSymmetric(root.left):
            #return False
def BuildTree(values):
    if not values:
        return None
    root=TreeNode(values[0])
    #出队
    queue=deque([root])
    i=1
    while queue and i < len(values):
        current = queue.popleft()
        #左节点入队
        if i < len(values) and values[i] is not None:
            current.left=TreeNode(values[i])
            queue.append(current.left)
        i+=1
        #右节点入队
        if i < len(values) and values[i] is not None:
            current.right=TreeNode(values[i])
            queue.append(current.right)
        i+=1
    return root
rootvalues=[1,2,2,3,4,4,3]
root=BuildTree(rootvalues)
print(f"输入：root = {rootvalues}， 输出：{IsSymmetric(root)}")
rootvalues2=[1,2,2,None,3,None,3]
root=BuildTree(rootvalues2)
print(f"输入：root = {rootvalues2}， 输出：{IsSymmetric(root2)}")

输入：root = [1, 2, 2, 3, 4, 4, 3]， 输出：True
输入：root = [1, 2, 2, None, 3, None, 3]， 输出：False


## 问题 11  

给你一个由 '1'（陆地）和 '0'（水）组成的的二维网格，请你计算网格中岛屿的数量。岛屿总是被水包围，并且每座岛屿只能由水平方向和/或竖直方向上相邻的陆地连接形成。

此外，你可以假设该网格的四条边均被水包围。

分析算法设计思路，计算时间复杂度，并基于python编程实现

e.g.:
---
1.
输入：grid = [

    ["1","1","1","1","0"],

    ["1","1","0","1","0"], 

    ["1","1","0","0","0"], 

    ["0","0","0","0","0"]  

]

输出：1

2.
输入：grid = [

    ["1","1","0","0","0"],

    ["1","1","0","0","0"],

    ["0","0","1","0","0"],

    ["0","0","0","1","1"]

]

输出：3

idea:   
岛屿是从1开始为中心 如果水平竖直碰见了1就被归化成同一座岛屿直至上下左右都为0时这个岛屿结束为一个整体 最后找到所有的岛屿      
因为和1相连的陆地我们视作归并为同一个岛屿 所以我们到时候把上下左右为1的是置为0   
从网格的第一个位置开始grid[0][0] 如果为1则去上下左右延申 把是1的置为0视作扩大成一个大岛(注意边界) 如果为0就跳过 到grid[0][1]   
采用一边访问一边扩散岛屿的方法 访问到为1则count+1并把临近为1的置为0视作同个岛屿 最后访问完整个岛屿 看最终count   
关于时间复杂度，因为要访问完整个岛屿 若m行n列则时间复杂度为O(m*n) 两层嵌套   

In [58]:
# add your code here
# your algorithm time complexity is:O(m*n)
def Countlands(grid):
    if not grid or not grid[0]:
        return 0
    #需要遍历整个网格 为俩个嵌套列表 行
    rows=len(grid)
    #列数为里面的列表中的元素数量 列
    cols=len(grid[0])
    count = 0
    #深度优先搜索去处理grid
    def dfs(r, c):
        #如果出界或者不是陆地 注意原列表 '1' '0'表示的 这里判断也要''
        if r<0 or r>=rows or c<0 or c>=cols or grid[r][c]!='1':
            #不作处理延伸不了 直接返回即可
            return
        #标记当前为已访问
        grid[r][c]='0'
        #向横竖四个方向扩展 上下左右
        dfs(r-1,c)  
        dfs(r+1,c)  
        dfs(r,c-1)  
        dfs(r,c+1)
    #遍历网格
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == '1':
                count += 1
                dfs(r, c)  #开始淹没整座岛屿

    return count
grid = [
["1","1","1","1","0"],

["1","1","0","1","0"],

["1","1","0","0","0"],

["0","0","0","0","0"]
]
print(grid) 
print('岛屿数目为：',Countlands(grid)) 
grid = [

["1","1","0","0","0"],

["1","1","0","0","0"],

["0","0","1","0","0"],

["0","0","0","1","1"]

]
print(grid) 
print('岛屿数目为：',Countlands(grid)) 

[['1', '1', '1', '1', '0'], ['1', '1', '0', '1', '0'], ['1', '1', '0', '0', '0'], ['0', '0', '0', '0', '0']]
岛屿数目为： 1
[['1', '1', '0', '0', '0'], ['1', '1', '0', '0', '0'], ['0', '0', '1', '0', '0'], ['0', '0', '0', '1', '1']]
岛屿数目为： 3
